In [1]:
%cd ..
%pwd

/workspace/DeepSpeech-2.x


'/workspace/DeepSpeech-2.x'

In [2]:
import math
import random
import tarfile
import logging
import numpy as np
from collections import namedtuple
from functools import partial

import paddle
from paddle.io import Dataset
from paddle.io import DataLoader
from paddle.io import BatchSampler
from paddle.io import DistributedBatchSampler
from paddle import distributed as dist



/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


In [3]:
import sys
import argparse
import functools
from deepspeech.utils.utility import add_arguments, print_arguments
parser = argparse.ArgumentParser(description=__doc__)
add_arg = functools.partial(add_arguments, argparser=parser)
# yapf: disable
add_arg('num_samples',      int,    5,     "# of samples to infer.")
add_arg('beam_size',        int,    500,    "Beam search width.")
add_arg('num_proc_bsearch', int,    8,      "# of CPUs for beam search.")
add_arg('num_conv_layers',  int,    2,      "# of convolution layers.")
add_arg('num_rnn_layers',   int,    3,      "# of recurrent layers.")
add_arg('rnn_layer_size',   int,    2048,   "# of recurrent cells per layer.")
add_arg('alpha',            float,  2.5,    "Coef of LM for beam search.")
add_arg('beta',             float,  0.3,    "Coef of WC for beam search.")
add_arg('cutoff_prob',      float,  1.0,    "Cutoff probability for pruning.")
add_arg('cutoff_top_n',     int,    40,     "Cutoff number for pruning.")
add_arg('use_gru',          bool,   False,  "Use GRUs instead of simple RNNs.")
add_arg('use_gpu',          bool,   True,   "Use GPU or not.")
add_arg('share_rnn_weights',bool,   True,   "Share input-hidden weights across "
                                            "bi-directional RNNs. Not for GRU.")
add_arg('unit_type',    str,
        'char',
        "Options: char, word, spm.",
        choices=['char', 'word', 'spm'])
add_arg('spm_model_prefix',    str,
        'examples/tiny/s1/data/spm_bpe',
        "spm model prefix.",)
add_arg('infer_manifest',   str,
        'examples/tiny/s1/data/manifest.tiny',
        "Filepath of manifest to infer.")
add_arg('mean_std_path',    str,
        'examples/tiny/s1/data/mean_std.npz',
        "Filepath of normalizer's mean & std.")
add_arg('vocab_path',       str,
        'examples/tiny/s1/data/vocab.txt',
        "Filepath of vocabulary.")
add_arg('lang_model_path',  str,
        'models/lm/common_crawl_00.prune01111.trie.klm',
        "Filepath for language model.")
add_arg('model_path',       str,
        'examples/tiny/s1/checkpoints/step_final',
        "If None, the training starts from scratch, "
        "otherwise, it resumes from the pre-trained model.")
add_arg('decoding_method',  str,
        'ctc_beam_search',
        "Decoding method. Options: ctc_beam_search, ctc_greedy",
        choices = ['ctc_beam_search', 'ctc_greedy'])
add_arg('error_rate_type',  str,
        'wer',
        "Error rate type for evaluation.",
        choices=['wer', 'cer'])
add_arg('specgram_type',    str,
        'fbank',
        "Audio feature type. Options: linear, mfcc.",
        choices=['linear', 'mfcc'])
add_arg('feat_dim',  int,  80, "mfcc or fbank feat dim.")
add_arg('delta_delta',  bool,  False, "delta delta")
# yapf: disable
args = parser.parse_args([])
print(vars(args))

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[WARNING 2021/04/16 06:32:09 __init__.py:93] register user softmax to paddle, remove this when fixed!
[WARNING 2021/04/16 06:32:09 __init__.py:97] register user log_softmax to paddle, remove this when fixed!
[WARNING 2021/04/16 06:32:09 __init__.py:101] register user sigmoid to paddle, remove this when fixed!
[WARNING 2021/04/16 06:32:09 __init__.py:105] register user log_sigmoid to paddle, remove this when fixed!
[WARNING 2021/04/16 06:32:09 __init__.py:109] register user relu to paddle, remove this when fixed!
[WARNING 2021/04/16 06:32:09 __init__.py:119] override cat of paddle if exists or register, remove thi

{'num_samples': 5, 'beam_size': 500, 'num_proc_bsearch': 8, 'num_conv_layers': 2, 'num_rnn_layers': 3, 'rnn_layer_size': 2048, 'alpha': 2.5, 'beta': 0.3, 'cutoff_prob': 1.0, 'cutoff_top_n': 40, 'use_gru': False, 'use_gpu': True, 'share_rnn_weights': True, 'unit_type': 'char', 'spm_model_prefix': 'examples/tiny/s1/data/spm_bpe', 'infer_manifest': 'examples/tiny/s1/data/manifest.tiny', 'mean_std_path': 'examples/tiny/s1/data/mean_std.npz', 'vocab_path': 'examples/tiny/s1/data/vocab.txt', 'lang_model_path': 'models/lm/common_crawl_00.prune01111.trie.klm', 'model_path': 'examples/tiny/s1/checkpoints/step_final', 'decoding_method': 'ctc_beam_search', 'error_rate_type': 'wer', 'specgram_type': 'fbank', 'feat_dim': 80, 'delta_delta': False}


In [4]:
import sys
import argparse
import functools
from deepspeech.utils.utility import add_arguments, print_arguments
parser = argparse.ArgumentParser(description=__doc__)
add_arg = functools.partial(add_arguments, argparser=parser)
# yapf: disable
add_arg('num_samples',      int,    5,     "# of samples to infer.")
add_arg('beam_size',        int,    500,    "Beam search width.")
add_arg('num_proc_bsearch', int,    8,      "# of CPUs for beam search.")
add_arg('num_conv_layers',  int,    2,      "# of convolution layers.")
add_arg('num_rnn_layers',   int,    3,      "# of recurrent layers.")
add_arg('rnn_layer_size',   int,    2048,   "# of recurrent cells per layer.")
add_arg('alpha',            float,  2.5,    "Coef of LM for beam search.")
add_arg('beta',             float,  0.3,    "Coef of WC for beam search.")
add_arg('cutoff_prob',      float,  1.0,    "Cutoff probability for pruning.")
add_arg('cutoff_top_n',     int,    40,     "Cutoff number for pruning.")
add_arg('use_gru',          bool,   False,  "Use GRUs instead of simple RNNs.")
add_arg('use_gpu',          bool,   True,   "Use GPU or not.")
add_arg('share_rnn_weights',bool,   True,   "Share input-hidden weights across "
                                            "bi-directional RNNs. Not for GRU.")
add_arg('unit_type',    str,
        'char',
        "Options: char, word, spm.",
        choices=['char', 'word', 'spm'])
add_arg('spm_model_prefix',    str,
        'examples/aishell/s1/data/spm_bpe',
        "spm model prefix.",)
add_arg('infer_manifest',   str,
        'examples/aishell/s1/data/manifest.test',
        "Filepath of manifest to infer.")
add_arg('mean_std_path',    str,
        '',
        "examples/aishell/s1/data/mean_std.npz, Filepath of normalizer's mean & std.")
add_arg('vocab_path',       str,
        'examples/aishell/s1/data/vocab.txt',
        "Filepath of vocabulary.")
add_arg('lang_model_path',  str,
        'models/lm/common_crawl_00.prune01111.trie.klm',
        "Filepath for language model.")
add_arg('model_path',       str,
        'examples/aishell/s1/checkpoints/step_final',
        "If None, the training starts from scratch, "
        "otherwise, it resumes from the pre-trained model.")
add_arg('decoding_method',  str,
        'ctc_beam_search',
        "Decoding method. Options: ctc_beam_search, ctc_greedy",
        choices = ['ctc_beam_search', 'ctc_greedy'])
add_arg('error_rate_type',  str,
        'wer',
        "Error rate type for evaluation.",
        choices=['wer', 'cer'])
add_arg('specgram_type',    str,
        'fbank',
        "Audio feature type. Options: linear, mfcc.",
        choices=['linear', 'mfcc', 'fbank'])
add_arg('feat_dim',  int,  80, "mfcc or fbank feat dim.")
add_arg('delta_delta',  bool,  False, "delta delta")
# yapf: disable
args = parser.parse_args([])
print(vars(args))

{'num_samples': 5, 'beam_size': 500, 'num_proc_bsearch': 8, 'num_conv_layers': 2, 'num_rnn_layers': 3, 'rnn_layer_size': 2048, 'alpha': 2.5, 'beta': 0.3, 'cutoff_prob': 1.0, 'cutoff_top_n': 40, 'use_gru': False, 'use_gpu': True, 'share_rnn_weights': True, 'unit_type': 'char', 'spm_model_prefix': 'examples/aishell/s1/data/spm_bpe', 'infer_manifest': 'examples/aishell/s1/data/manifest.test', 'mean_std_path': '', 'vocab_path': 'examples/aishell/s1/data/vocab.txt', 'lang_model_path': 'models/lm/common_crawl_00.prune01111.trie.klm', 'model_path': 'examples/aishell/s1/checkpoints/step_final', 'decoding_method': 'ctc_beam_search', 'error_rate_type': 'wer', 'specgram_type': 'fbank', 'feat_dim': 80, 'delta_delta': False}


In [5]:
from deepspeech.frontend.utility import read_manifest
from deepspeech.frontend.augmentor.augmentation import AugmentationPipeline
from deepspeech.frontend.featurizer.speech_featurizer import SpeechFeaturizer
from deepspeech.frontend.speech import SpeechSegment
from deepspeech.frontend.normalizer import FeatureNormalizer


from deepspeech.io.collator import SpeechCollator
from deepspeech.io.dataset import ManifestDataset
from deepspeech.io.sampler import (
    SortagradDistributedBatchSampler,
    SortagradBatchSampler,
)
from deepspeech.io import create_dataloader
batch_reader = create_dataloader(
                      manifest_path=args.infer_manifest,
                      unit_type=args.unit_type,
                      vocab_filepath=args.vocab_path,
                      mean_std_filepath=args.mean_std_path,
                      spm_model_prefix=args.spm_model_prefix,
                      augmentation_config='{}',
                      max_input_len=27.0,
                      min_input_len=0.0,
                      max_output_len=float('inf'),
                      min_output_len=0.0,
                      max_output_input_ratio=float('inf'),
                      min_output_input_ratio=0.0,
                      stride_ms=10.0,
                      window_ms=20.0,
                      max_freq=None,
                      specgram_type=args.specgram_type,
                      feat_dim=args.feat_dim,
                      delta_delta=args.delta_delta,
                      use_dB_normalization=True,
                      random_seed=0,
                      keep_transcription_text=True,
                      is_training=False,
                      batch_size=args.num_samples,
                      num_workers=0,
                      sortagrad=True,
                      shuffle_method=None,
                      dist=False)

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/scipy/fftpack/__init__.py:103: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,
/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/numba/core/

In [6]:
for idx, (audio, audio_len, text, text_len) in enumerate(batch_reader()):
    print('test:', text)
    print("test raw:", ''.join( chr(i) for i in text[0][:int(text_len[0])] ))
    print("test raw:", ''.join( chr(i) for i in text[-1][:int(text_len[-1])] ))
    print('audio len:', audio_len)
    print('test len:', text_len)
    print('audio:', audio)
    break

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


fbank
[232 387 331 ... 249 249 262] int16
fbank
[-138 -219 -192 ...  338  324  351] int16
fbank
[ 694 1175 1022 ...  553  514  627] int16
fbank
[-39 -79 -53 ... 139 172  99] int16
fbank
[-277 -480 -425 ...  758  767  739] int16
fbank
[ 399  693  609 ... 1291 1270 1291] int16


/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:354: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if arr.dtype == np.object:


fbank
[ -750 -1254 -1107 ...  2276  1889  2067] int16
fbank
[ -127  -199  -149 ... -5243 -5065 -5398] int16
fbank
[ 465  783  677 ...  980  903 1008] int16
fbank
[ 90 160 157 ...  -2 -16 -21] int16
fbank
[ 213  345  295 ... 2483 2246 2501] int16
fbank
[ -86 -159 -131 ...  270  258  290] int16
fbank
[-1023 -1714 -1505 ...  1532  1596  1575] int16
fbank
[-366 -602 -527 ...  374  370  379] int16
fbank
[ 761 1275 1127 ...  369  413  295] int16
fbank
[382 621 550 ... 161 161 174] int16
fbank
[ -28  -91 -120 ...   28   34   11] int16
fbank
[ -5  -5  -5 ... 268 294 341] int16
fbank
[240 417 684 ... 267 262 219] int16
fbank
[131 206 194 ... 383 320 343] int16
test: Tensor(shape=[5, 7], dtype=int32, place=CUDAPinnedPlace, stop_gradient=True,
       [[31069, 21487, 29233, 30340, 20320, -1   , -1   ],
        [20540, 24471, 19968, 25552, 30340, 26159, -1   ],
        [36825, 20010, 31243, 24230, 26159, 32654, 30340],
        [20108, 21040, 20108, -1   , -1   , -1   , -1   ],
        [21435, 34892

In [7]:
keep_transcription_text=False
batch_reader = create_dataloader(
                      manifest_path=args.infer_manifest,
                      unit_type=args.unit_type,
                      vocab_filepath=args.vocab_path,
                      mean_std_filepath=args.mean_std_path,
                      spm_model_prefix=args.spm_model_prefix,
                      augmentation_config='{}',
                      max_input_len=27.0,
                      min_input_len=0.0,
                      max_output_len=float('inf'),
                      min_output_len=0.0,
                      max_output_input_ratio=float('inf'),
                      min_output_input_ratio=0.0,
                      stride_ms=10.0,
                      window_ms=20.0,
                      max_freq=None,
                      specgram_type=args.specgram_type,
                      feat_dim=args.feat_dim,
                      delta_delta=args.delta_delta,
                      use_dB_normalization=True,
                      random_seed=0,
                      keep_transcription_text=keep_transcription_text,
                      is_training=False,
                      batch_size=args.num_samples,
                      num_workers=0,
                      sortagrad=True,
                      shuffle_method=None,
                      dist=False)
for idx, (audio, audio_len, text, text_len) in enumerate(batch_reader()):
    print('test:', text)
    print("test raw:", ''.join( chr(i) for i in text[0][:int(text_len[0])] ))
    print("test raw:", ''.join( chr(i) for i in text[-1][:int(text_len[-1])] ))
    print('test len:', text_len)
    print('audio:', audio)
    print('audio len:', audio_len)
    break

fbank
[232 387 331 ... 249 249 262] int16
fbank
[-138 -219 -192 ...  338  324  351] int16
fbank
[ 694 1175 1022 ...  553  514  627] int16
fbank
[-39 -79 -53 ... 139 172  99] int16
fbank
[-277 -480 -425 ...  758  767  739] int16
fbank
test: Tensor(shape=[5, 7], dtype=int32, place=CUDAPinnedPlace, stop_gradient=True,
       [[2695,  505, 2332, 2553,  169, -1  , -1  ],
        [ 230, 1237,  2  , 1556, 2553, 1694, -1  ],
        [3703,  28 , 2739, 1172, 1694, 2966, 2553],
        [ 70 ,  355,  70 , -1  , -1  , -1  , -1  ],
        [ 477, 3363, 1621,  412, -1  , -1  , -1  ]])
[ 399  693  609 ... 1291 1270 1291] int16
test raw: ઇǹज৹©
test raw: ǝണٕƜ
test len: Tensor(shape=[5], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [5, 6, 7, 3, 4])
audio: Tensor(shape=[5, 207, 80], dtype=float32, place=CUDAPinnedPlace, stop_gradient=True,
       [[[12.25794601, 12.61855793, 10.37306023, ..., 13.12571049, 11.53678799, 10.32210350],
         [13.32333183, 13.41336918, 11.44248962, ..., 13.6

In [8]:
import sys
import argparse
import functools
from deepspeech.utils.utility import add_arguments, print_arguments
parser = argparse.ArgumentParser(description=__doc__)
add_arg = functools.partial(add_arguments, argparser=parser)

add_arg('num_samples',      int,    1,    "# of samples to for statistics.")
add_arg('specgram_type',    str,   'fbank',
        "Audio feature type. Options: linear, mfcc, fbank.",
        choices=['linear', 'mfcc', 'fbank'])
add_arg('feat_dim',    int, 80, "Audio feature dim.")
add_arg('delta_delta',    bool, False,"Audio feature with delta delta.")
add_arg('stride_ms',    float, 10.0,  "stride length in ms.")
add_arg('window_ms',    float, 25.0,  "stride length in ms.")
add_arg('sample_rate',    int, 16000,  "target sample rate.")
add_arg('manifest_path',    str,
        'examples/aishell/s1/data/manifest.train',
        "Filepath of manifest to compute normalizer's mean and stddev.")
add_arg('output_path',    str,
        'examples/aishell/s1/data/mean_std.npz',
        "Filepath of write mean and stddev to (.npz).")
args = parser.parse_args([])
print(vars(args))


{'num_samples': 1, 'specgram_type': 'fbank', 'feat_dim': 80, 'delta_delta': False, 'stride_ms': 10.0, 'window_ms': 25.0, 'sample_rate': 16000, 'manifest_path': 'examples/aishell/s1/data/manifest.train', 'output_path': 'examples/aishell/s1/data/mean_std.npz'}


In [9]:

from deepspeech.frontend.augmentor.augmentation import AugmentationPipeline
from deepspeech.frontend.featurizer.audio_featurizer import AudioFeaturizer
from deepspeech.frontend.normalizer import FeatureNormalizer
from deepspeech.frontend.audio import AudioSegment
from deepspeech.frontend.utility import load_cmvn
from deepspeech.frontend.utility import read_manifest



def mean(args):
    augmentation_pipeline = AugmentationPipeline('{}')
    audio_featurizer = AudioFeaturizer(
        specgram_type=args.specgram_type,
        feat_dim=args.feat_dim,
        delta_delta=args.delta_delta,
        stride_ms=args.stride_ms,
        window_ms=args.window_ms,
        n_fft=None,
        max_freq=None,
        target_sample_rate=args.sample_rate,
        use_dB_normalization=True,
        target_dB=-20,
        dither=0.0)

    def augment_and_featurize(audio_segment):
        augmentation_pipeline.transform_audio(audio_segment)
        return audio_featurizer.featurize(audio_segment)

    normalizer = FeatureNormalizer(
        mean_std_filepath=None,
        manifest_path=args.manifest_path,
        featurize_func=augment_and_featurize,
        num_samples=args.num_samples)
    normalizer.write_to_file(args.output_path)



In [16]:
wav='/workspace/DeepSpeech-2.x/examples/aishell/s1/../../..//examples/dataset/aishell/data_aishell/wav/test/S0916/BAC009S0916W0426.wav'
test='祝可爱的你'
audio_featurizer = AudioFeaturizer(
        specgram_type=args.specgram_type,
        feat_dim=args.feat_dim,
        delta_delta=args.delta_delta,
        stride_ms=args.stride_ms,
        window_ms=args.window_ms,
        n_fft=None,
        max_freq=None,
        target_sample_rate=args.sample_rate,
        use_dB_normalization=False,
        target_dB=-20,
        dither=0.0)
samples = AudioSegment.from_file(wav)
print(samples._samples)
print(samples._samples * 2**15)
print(len(samples._samples))
feat = audio_featurizer.featurize(samples, False, False)
feat = feat.T
print(feat.shape, feat.dtype)
print(feat)

from python_speech_features import logfbank
max_freq = args.sample_rate / 2
fbank_feat = logfbank(
            signal=samples.to('int16'),
            samplerate=args.sample_rate,
            winlen=0.001 * args.window_ms,
            winstep=0.001 * args.stride_ms,
            nfilt=args.feat_dim,
            nfft=512,
            lowfreq=20,
            highfreq=max_freq,
            preemph=0.97,
            dither=0.0,
            wintype='povey')
print(fbank_feat.shape, fbank_feat.dtype)
print(fbank_feat)

[0.00164795 0.00274658 0.00234985 ... 0.00177002 0.00177002 0.00186157]
[54. 90. 77. ... 58. 58. 61.]
29746
fbank
[54 90 77 ... 58 58 61] int16
(184, 80) float64
[[10.61737914 10.07708936  5.32487528 ... 10.2481839   8.89699394
   7.80671114]
 [11.0440077  10.3180721   6.30866128 ... 11.23730926 10.35838868
   8.83860079]
 [10.26930555  9.99636567  7.3296638  ... 10.45131595  9.69295303
   7.96168491]
 ...
 [10.14497345  9.88674207  6.73801138 ... 10.21580627  9.00343472
   8.75616521]
 [ 9.97745961  9.67949736  7.90660425 ... 10.22436653  9.59456493
   7.69287184]
 [ 6.47357374  7.76335491  7.75765843 ...  9.96522077  9.6226365
   8.16007108]]
(184, 80) float64
[[10.61737914 10.07708936  5.32487528 ... 10.2481839   8.89699394
   7.80671114]
 [11.0440077  10.3180721   6.30866128 ... 11.23730926 10.35838868
   8.83860079]
 [10.26930555  9.99636567  7.3296638  ... 10.45131595  9.69295303
   7.96168491]
 ...
 [10.14497345  9.88674207  6.73801138 ... 10.21580627  9.00343472
   8.75616521]


In [17]:
a = np.hstack([feat, feat])
print(a.shape)
m = np.mean(a, axis=1)
print(m)
print(m.shape)
std = np.std(a, axis=1)
print(std.shape)
print(std)

(184, 160)
[ 8.59522397  8.43148278  8.36414052  8.45487173  8.31761643  8.04843683
  8.01683696  7.6574614   7.95521932  8.22945157 10.20138275  9.0447775
  9.14763398  9.18184349  9.03801065  9.04852307  8.67706728  8.71894271
  9.54553655  9.19535135  8.76413076  8.47828946  8.52586143  8.49469288
  8.72461247  8.28562879  8.11581393  7.99922156  7.91023364  8.04142296
  7.89762773  7.76257636  8.32043745  8.01592886  8.34109665  8.90115454
  8.48246945  7.98658664  8.05745122  8.11384088  8.18864479  8.8091827
 11.8067711  13.25258218 14.44311795 13.90515283 14.00120623 13.99801252
 13.81595394 13.6379904  13.3574897  13.14933334 12.96518543 13.02601156
 12.70246737 12.54410834 12.15615068 11.86574681 11.67497882 10.79645481
 10.48150035 10.03758575 10.05637027  9.92891308 10.06923218 12.43382431
 12.71428321 14.33135052 13.94470959 14.29188291 14.11483993 14.03496606
 13.78167331 13.66701466 14.40308625 14.73934137 15.09569382 14.89565815
 15.10519995 14.94383582 15.03275563 15.42

In [18]:
import torch
import torchaudio
import torchaudio.compliance.kaldi as kaldi
import torchaudio.sox_effects as sox_effects
from torch.nn.utils.rnn import pad_sequence
torchaudio.set_audio_backend("sox")

In [19]:
waveform, sample_rate = torchaudio.load_wav(wav)
print(waveform.shape)
print(waveform)
mat = kaldi.fbank(
                waveform,
                num_mel_bins=80,
                frame_length=25,
                frame_shift=10,
                dither=0,
                energy_floor=0.0,
                sample_frequency=sample_rate
            )
mat = mat.detach().numpy()
print(mat.shape)
print(mat)

print('-----------')
print(samples._samples)
aud = torch.tensor(samples._samples).view(1, -1)
mat = kaldi.fbank(
                aud,
                num_mel_bins=80,
                frame_length=25,
                frame_shift=10,
                dither=0,
                energy_floor=0.0,
                sample_frequency=sample_rate
            )
mat = mat.detach().numpy()
print(mat.shape)
print(mat)

print('**************')
print(samples._samples)
tmp = samples.to('int16').astype('float32')
print(tmp, tmp.dtype)
aud = torch.tensor(tmp).view(1, -1)
mat = kaldi.fbank(
                aud,
                num_mel_bins=80,
                frame_length=25,
                frame_shift=10,
                dither=0,
                energy_floor=0.0,
                sample_frequency=sample_rate
            )
mat = mat.detach().numpy()
print(mat.shape)
print(mat)

torch.Size([1, 29746])
tensor([[54., 90., 77.,  ..., 58., 58., 61.]])
(184, 80)
[[10.617376  10.077089   5.3248763 ... 10.248186   8.896992   7.8067265]
 [11.044004  10.318072   6.3086634 ... 11.237308  10.358393   8.838616 ]
 [10.269302   9.9963665  7.3296647 ... 10.451319   9.692951   7.9617033]
 ...
 [10.14497    9.886743   6.738012  ... 10.215809   9.0034275  8.756177 ]
 [ 9.977456   9.679498   7.9066052 ... 10.224365   9.594568   7.6928873]
 [ 6.4735703  7.7633557  7.7576594 ...  9.965221   9.622637   8.160085 ]]
-----------
[0.00164795 0.00274658 0.00234985 ... 0.00177002 0.00177002 0.00186157]
(184, 80)
[[-10.177039 -10.717326 -15.46954  ... -10.546229 -11.897424 -12.987689]
 [ -9.750411 -10.476343 -14.485752 ...  -9.557108 -10.436023 -11.955799]
 [-10.525113 -10.798049 -13.46475  ... -10.343097 -11.101464 -12.832712]
 ...
 [-10.649446 -10.907673 -14.056403 ... -10.578607 -11.790988 -12.038239]
 [-10.816959 -11.114918 -12.88781  ... -10.570049 -11.199847 -13.101528]
 [-14.320845

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: torchaudio.backend.sox_backend.load_wav has been deprecated and will be removed from 0.9.0 release. Please use "torchaudio.load".
  """Entry point for launching an IPython kernel.


In [20]:
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as iowav

(rate,sig) = iowav.read(wav)
print(sig.shape)
print(sig)

# note that generally nfilt=40 is used for speech recognition
fbank_feat = logfbank(sig,nfilt=80,lowfreq=20,dither=0,wintype='povey')
print(fbank_feat.shape)
print(fbank_feat)

# the computed fbank coefficents of english.wav with dimension [110,23]
# [ 12.2865	12.6906	13.1765	15.714	16.064	15.7553	16.5746	16.9205	16.6472	16.1302	16.4576	16.7326	16.8864	17.7215	18.88	19.1377	19.1495	18.6683	18.3886	20.3506	20.2772	18.8248	18.1899
# 11.9198	13.146	14.7215	15.8642	17.4288	16.394	16.8238	16.1095	16.4297	16.6331	16.3163	16.5093	17.4981	18.3429	19.6555	19.6263	19.8435	19.0534	19.001	20.0287	19.7707	19.5852	19.1112
# ...
# ...
# the same with that using kaldi commands: compute-fbank-feats --dither=0.0

mfcc_feat = mfcc(sig,dither=0,useEnergy=True,wintype='povey')
print(mfcc_feat.shape)
print(mfcc_feat)

# the computed mfcc coefficents of english.wav with dimension [110,13]
# [ 17.1337	-23.3651	-7.41751	-7.73686	-21.3682	-8.93884	-3.70843	4.68346	-16.0676	12.782	-7.24054	8.25089	10.7292
# 17.1692	-23.3028	-5.61872	-4.0075	-23.287	-20.6101	-5.51584	-6.15273	-14.4333	8.13052	-0.0345329	2.06274	-0.564298
# ...
# ...
# the same with that using kaldi commands: compute-mfcc-feats --dither=0.0

(29746,)
[54 90 77 ... 58 58 61]
(184, 80)
[[10.61737914 10.07708936  5.32487528 ... 10.2481839   8.89699394
   7.80671114]
 [11.0440077  10.3180721   6.30866128 ... 11.23730926 10.35838868
   8.83860079]
 [10.26930555  9.99636567  7.3296638  ... 10.45131595  9.69295303
   7.96168491]
 ...
 [10.14497345  9.88674207  6.73801138 ... 10.21580627  9.00343472
   8.75616521]
 [ 9.97745961  9.67949736  7.90660425 ... 10.22436653  9.59456493
   7.69287184]
 [ 6.47357374  7.76335491  7.75765843 ...  9.96522077  9.6226365
   8.16007108]]
(184, 13)
[[ 14.73775998 -13.30393391   5.85974818 ...  -3.42359739   2.82785335
    8.86862748]
 [ 15.31274834 -13.33671651   4.06537223 ...   8.15970347   2.15934846
    6.78353115]
 [ 13.82218765 -13.39296404   6.8304843  ...   2.55332563   8.86724453
   -0.05919222]
 ...
 [ 13.5837844  -13.42104892  11.21222354 ...   4.81477718   1.66627505
    5.59045842]
 [ 13.75757034 -13.92626662  13.06074011 ...  -0.46694046   5.56214833
   12.0785146 ]
 [ 11.92813809 -

In [21]:
fbank_feat = logfbank(samples._samples,nfilt=80,lowfreq=20,dither=0,wintype='povey')
print(fbank_feat.shape)
print(fbank_feat)

mfcc_feat = mfcc(samples._samples,dither=0,useEnergy=True,wintype='povey')
print(mfcc_feat.shape)
print(mfcc_feat)

(184, 80)
[[-10.17703627 -10.71732606 -15.46954014 ... -10.54623152 -11.89742148
  -12.98770428]
 [ -9.75040771 -10.47634331 -14.48575413 ...  -9.55710616 -10.43602673
  -11.95581463]
 [-10.52510987 -10.79804975 -13.46475161 ... -10.34309947 -11.10146239
  -12.83273051]
 ...
 [-10.64944197 -10.90767335 -14.05640404 ... -10.57860915 -11.7909807
  -12.03825021]
 [-10.8169558  -11.11491806 -12.88781116 ... -10.57004889 -11.19985048
  -13.10154358]
 [-14.32084168 -13.03106051 -13.03675699 ... -10.82919465 -11.17177892
  -12.63434434]]
(184, 13)
[[ -6.05665544 -13.30393391   5.85974818 ...  -3.42359739   2.82785335
    8.86862748]
 [ -5.48166707 -13.33671651   4.06537223 ...   8.15970347   2.15934846
    6.78353115]
 [ -6.97222776 -13.39296404   6.8304843  ...   2.55332563   8.86724453
   -0.05919222]
 ...
 [ -7.21063102 -13.42104892  11.21222354 ...   4.81477718   1.66627505
    5.59045842]
 [ -7.03684508 -13.92626662  13.06074011 ...  -0.46694046   5.56214833
   12.0785146 ]
 [ -8.8662773